In [2]:
import torch
import os

# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:50'
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [11]:
# !pip install diffusers transformers scipy
# !pip install accelerate

from diffusers import ControlNetModel, StableDiffusionControlNetPipeline, AutoencoderKL, StableDiffusionControlNetImg2ImgPipeline

device = mps_device

BASE_MODELS = [
    "SG161222/Realistic_Vision_V5.1_noVAE",
    "runwayml/stable-diffusion-v1-5"
]

# pre-trained vae
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16)

# pre-trained controlnet
controlnet = ControlNetModel.from_pretrained("monster-labs/control_v1p_sd15_qrcode_monster", torch_dtype=torch.float16)

# Load the pipeline and move it to the TPU
pipeline = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODELS[0],
    controlnet=controlnet,
    vae=vae,
    safety_checker=None,
    torch_dtype=torch.float16
).to(device)

image_pipe = StableDiffusionControlNetImg2ImgPipeline(**pipeline.components)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

TypeError: __init__() got an unexpected keyword argument 'image_encoder'

In [1]:
import requests
from PIL import Image
from io import BytesIO

url = 'https://i.ibb.co/0mrC3nH/IMG-7742.jpg'
response = requests.get(url)
image = Image.open(BytesIO(response.content))

In [8]:
prompt = "A small village on christmas eve"

output = pipeline(prompt, image)

  0%|          | 0/50 [00:00<?, ?it/s]

In [10]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
output.images[0].save(f"outputs/output_{current_time}.png")